# base line

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
(train_data, train_labels), _ = imdb.load_data(num_words=10000)

def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results
train_data = vectorize_sequences(train_data)

model = keras.Sequential([
    layers.Dense(16, activation="relu"),
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])
model.compile(optimizer="rmsprop",
             loss="binary_crossentropy",
             metrics=["accuracy"])
history_original = model.fit(train_data, train_labels,
                            epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.6889 - loss: 0.6108 - val_accuracy: 0.8721 - val_loss: 0.4005
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.8936 - loss: 0.3482 - val_accuracy: 0.8847 - val_loss: 0.3116
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9115 - loss: 0.2570 - val_accuracy: 0.8916 - val_loss: 0.2810
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9395 - loss: 0.1971 - val_accuracy: 0.8853 - val_loss: 0.2804
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.9499 - loss: 0.1617 - val_accuracy: 0.8862 - val_loss: 0.2809
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9548 - loss: 0.1382 - val_accuracy: 0.8811 - val_loss: 0.2997
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.9655 - loss: 0.1182 - val_accuracy: 0.8820 - val_loss: 0.3087
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.9691 - loss: 0.1076 - val_accuracy: 0.8781 - v

## 사용자 정의 지표

In [16]:
from tensorflow.keras import backend as K

class F1Score(keras.metrics.Metric):
    def __init__(self, name="f1_score", **kwargs):
        super().__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name="tp", initializer="zeros")
        self.false_positives = self.add_weight(name="fp", initializer="zeros")
        self.false_negatives = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = K.cast(y_true, "float32")
        y_pred = K.round(y_pred)

        self.true_positives.assign_add(K.sum(K.cast(y_true * y_pred, "float32")))
        self.false_positives.assign_add(K.sum(K.cast((1 - y_true) * y_pred, "float32")))
        self.false_negatives.assign_add(K.sum(K.cast(y_true * (1 - y_pred), "float32")))

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + K.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + K.epsilon())

        f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())
        return f1_score

    def reset_state(self):
        self.true_positives.assign(0)
        self.false_positives.assign(0)
        self.false_negatives.assign(0)

In [17]:
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy", F1Score()])
n_history = model.fit(train_data, train_labels,
        epochs=20, batch_size=512, validation_split=0.4)

Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.9901 - f1_score: 0.5027 - loss: 0.0275 - val_accuracy: 0.9317 - val_f1_score: 0.5005 - val_loss: 0.3471
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step - accuracy: 0.9994 - f1_score: 0.5027 - loss: 0.0061 - val_accuracy: 0.9305 - val_f1_score: 0.5018 - val_loss: 0.3643
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9997 - f1_score: 0.4984 - loss: 0.0042 - val_accuracy: 0.9293 - val_f1_score: 0.5009 - val_loss: 0.3784
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9990 - f1_score: 0.5015 - loss: 0.0057 - val_accuracy: 0.9292 - val_f1_score: 0.5000 - val_loss: 0.3871
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 1.0000 - f1_score: 0.5036 - loss: 0.0026 - val_accuracy: 0.9278 - val_f1_score: 0.5000 - val_loss: 0.3996
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 1.0000 - f1_score: 0.5042 - loss: 0.0025 - val_accuracy: 0.9264 - val_f1_score: 0.4928 - val